In [1]:
import os
import openai
import dotenv
import pandas as pd
from IPython.display import display, Markdown

dotenv.load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
df = pd.read_parquet("./movie/data.dataset.parquet")
df.head()

,UserId,Movie Name,Rating
0,1,The Godfather (1972),10
1,21,The Godfather (1972),9
2,25,The Godfather (1972),9
3,27,The Godfather (1972),8
4,107,The Godfather (1972),10


In [3]:
user25 = df[df["UserId"] == 25]["Movie Name"][:-1].to_list()
user25_latest = df[df["UserId"] == 25]["Movie Name"][-1:].to_list()[0]
df[df["UserId"] == 25]

,UserId,Movie Name,Rating
2,25,The Godfather (1972),9
13622,25,Iron Man 3 (2013),8
49764,25,Kick-Ass 2 (2013),9
50455,25,The Godfather Part II (1974),10
50566,25,Scarface (1983),8
50651,25,The Godfather Part III (1990),8
50701,25,Coach Carter (2005),9
50729,25,Role Models (2008),8
50751,25,Argo (2012),9
51643,25,The Hunger Games (2012),7


In [4]:
user107 = df[df["UserId"] == 107]["Movie Name"][:-1].to_list()
user107_latest = df[df["UserId"] == 107]["Movie Name"][-1:].to_list()[0]
df[df["UserId"] == 107]

,UserId,Movie Name,Rating
4,107,The Godfather (1972),10
13630,107,Iron Man 3 (2013),8
16012,107,Zero Dark Thirty (2012),8
36963,107,Fast &amp; Furious 6 (2013),8
39897,107,The Avengers (2012),8
41282,107,The Hangover Part III (2013),6
57137,107,Olympus Has Fallen (2013),5
98936,107,The Amazing Spider-Man (2012),6
101352,107,Catch Me If You Can (2002),8
118911,107,Men of Honor (2000),9


In [5]:
ENGINE = "chat-gpt"
TEMPERATURE = 0.3
MAX_TOKENS = 2000
TOP_P = 1
FREQUENCY_PENALTY = 0.0
PRESENCE_PENALTY = 0.0

def run(user_msg: str, 
        system_msg: str = """You're going to recommend a movie to users based on their movie watch history.
    As a professional recommender system, you're going to provide the following steps:

    # Steps
    ## movie watch history
    [movie watch history]
    [Analyze the movie watch history by clustering the movies]

    ## Your favorite movie genre
    [Provide your favorite movie genre based on the clustering result above]

    ## movie recommendation
    [Write recommended movie name fist and then explain why you recommend the movie to the user]

    ## Safty
    Do not show sub title of the each stpes above""", 
    examples_user_prompt: str = """
    # movie watch history
    ['The Godfather (1972)', 'Iron Man 3 (2013)', 'Zero Dark Thirty (2012)', 'Fast &amp; Furious 6 (2013)', 'The Avengers (2012)', 'The Hangover Part III (2013)', 'Olympus Has Fallen (2013)', 'The Amazing Spider-Man (2012)', 'Catch Me If You Can (2002)', 'Men of Honor (2000)', 'The Blind Side (2009)', 'Undefeated (2011)']""", 
    examples_assistant_res: str = """
    ## Crime/Drama:
    * The Godfather (1972)
    * Catch Me If You Can (2002)
    * Men of Honor (2000)

    ## Action/Adventure:
    * Iron Man 3 (2013)
    * Fast & Furious 6 (2013)
    * The Avengers (2012)
    * Olympus Has Fallen (2013)
    * The Amazing Spider-Man (2012)

    ## Thriller:
    * Zero Dark Thirty (2012)

    ## Comedy:
    * The Hangover Part III (2013)

    ## Drama/Sports:
    * The Blind Side (2009)
    * Undefeated (2011)

    Your favorite geners are `Action/Adventure`, `Crime/Drama` because you watched the most movies in these genres.

    `Mad Max: Fury Road (2015)` 

    It's an action-packed, thrilling adventure film set in a post-apocalyptic world.
    The movie has received critical acclaim for its visuals, action sequences, and performances."""): 

    messages = [{"role":"system", "content":system_msg}, 
                {"role":"user","content":examples_user_prompt},
                {"role":"assistant","content":examples_assistant_res},
                {"role":"user","content":user_msg}
                ]

    res = openai.ChatCompletion.create(
        engine=ENGINE,
        messages = messages,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        top_p=TOP_P,
        frequency_penalty=FREQUENCY_PENALTY,
        presence_penalty=PRESENCE_PENALTY,
        n=1
        )
    
    return res.choices[0].message['content']


In [6]:
query = f"""
# movie watch history
{user25}
"""

Markdown(run(query))

    ## Crime/Drama:
    * The Godfather (1972)
    * The Godfather Part II (1974)
    * Scarface (1983)

    ## Action/Adventure:
    * Iron Man 3 (2013)
    * Kick-Ass 2 (2013)
    * Faster (2010)
    * The Hunger Games (2012)

    ## Comedy:
    * Role Models (2008)

    ## Drama/Sports:
    * Coach Carter (2005)

    ## Thriller:
    * Argo (2012)

    Your favorite genres are `Crime/Drama` and `Action/Adventure` because you have watched the most movies in these genres.

    `The Departed (2006)`

    It's a crime thriller film directed by Martin Scorsese. The movie has an incredible cast, including Leonardo DiCaprio, Matt Damon, and Jack Nicholson. The Departed has won several awards, including four Oscars, and is considered one of the best crime thrillers of all time. The movie has a similar tone and style to the movies you have watched in the past, making it a great recommendation for you.

In [7]:
system_msg = """
You're going to recommend a movie to users based on their movie watch history.
As a professional recommender system, you're going to provide the following steps:

# Steps
## movie watch history
[movie watch history]
[Analyze the movie watch history by clustering the movies]

# Recommendation Score
[If user ask for the recommendation score, provide the score, 1 is low and 10 is high chance to like the movie]
[Answer the question with the score first and then explain why you give the score in the next line]

## Safty
Do not show sub title of the each stpes above
"""

examples_user_prompt =f"""
# movie watch history
['The Godfather (1972)', 'Iron Man 3 (2013)', 'Zero Dark Thirty (2012)', 'Fast &amp; Furious 6 (2013)', 'The Avengers (2012)', 'The Hangover Part III (2013)', 'Olympus Has Fallen (2013)', 'The Amazing Spider-Man (2012)', 'Catch Me If You Can (2002)', 'Men of Honor (2000)', 'The Blind Side (2009)', 'Undefeated (2011)']
"""

examples_assistant_res = f"""
Recommendation Score: 6

I would recommend you watch The Bling Ring (2013) with a score of 6 out of 10.

The reason for the recommendation is that
"""

print("User watched", user25_latest, "and the rating was", df[df["UserId"]==25]["Rating"][-1:].to_list()[0])
print("---")
Markdown(run(system_msg=system_msg, examples_user_prompt=examples_user_prompt, examples_assistant_res=examples_assistant_res, user_msg=f"I watched {user25}. Would you recommend me the movie,{user25_latest}"))

User watched Chronicle (2012) and the rating was 7
---


Recommendation Score: 8

Based on your movie watch history, I would recommend you watch Chronicle (2012) with a score of 8 out of 10.

The reason for the recommendation is that Chronicle is a science-fiction film that explores the lives of three high school students who gain superpowers after encountering a mysterious object. It has elements of action, drama, and suspense, which are similar to the movies you have watched in the past, such as Kick-Ass 2 and Faster. Additionally, the movie has received positive reviews for its unique storytelling and impressive special effects.

In [8]:
df[df["UserId"]==25]

,UserId,Movie Name,Rating
2,25,The Godfather (1972),9
13622,25,Iron Man 3 (2013),8
49764,25,Kick-Ass 2 (2013),9
50455,25,The Godfather Part II (1974),10
50566,25,Scarface (1983),8
50651,25,The Godfather Part III (1990),8
50701,25,Coach Carter (2005),9
50729,25,Role Models (2008),8
50751,25,Argo (2012),9
51643,25,The Hunger Games (2012),7


In [9]:
def test_run(user_id: int = 14179):
    system_msg = """
    You're going to recommend a movie to users based on their movie watch history.
    As a professional personalization system, you're going to provide the following steps:

    # Steps
    ## 1. Analyze movie watch history
    [Add Genres to the movie watch history]

    ## 2. Calculate the weighted score for each genre
    
    ## 3. Cluster the movies based on the genres and weighted score

    ## 4. Predict User's Rating Score
    [Using the weighted scores calculated earlier]
    [When user ask for the recommendation score, provide the score, based on the rating history]
    [Answer the question with the score first and then explain why you give the score in the next line]

    # Safty
    Do not show sub title of the each stpes above
    """

    examples_user_prompt ="""
    I watched 
    [{'Movie Name': 'Dallas Buyers Club (2013)', 'Rating': 9},
    {'Movie Name': 'World Z (2013)', 'Rating': 7},
    {'Movie Name': 'Gravity (2013)', 'Rating': 9},
    {'Movie Name': 'Her (2013)', 'Rating': 10},
    {'Movie Name': '12 Years a Slave (2013)', 'Rating': 8},
    {'Movie Name': 'The Secret Life of Walter Mitty (2013)', 'Rating': 9},
    {'Movie Name': 'The Grand Budapest Hotel (2014)', 'Rating': 10},
    {'Movie Name': 'Riddick (2013)', 'Rating': 6},
    {'Movie Name': 'Kick-Ass 2 (2013)', 'Rating': 7},
    {'Movie Name': 'Star Trek Into Darkness (2013)', 'Rating': 7},
    {'Movie Name': 'The Wolf of Wall Street (2013)', 'Rating': 10},
    {'Movie Name': 'American Hustle (2013)', 'Rating': 7},
    {'Movie Name': 'Nebraska (2013)', 'Rating': 10},
    {'Movie Name': 'Inside Llewyn Davis (2013)', 'Rating': 10},
    {'Movie Name': 'R.I.P.D. (2013)', 'Rating': 4},
    {'Movie Name': 'The Book Thief (2013)', 'Rating': 9},
    {'Movie Name': 'Last Vegas (2013)', 'Rating': 5},
    {'Movie Name': 'The Purge (2013)', 'Rating': 3},
    {'Movie Name': 'Runner Runner (2013)', 'Rating': 7},
    {'Movie Name': 'The (2013)', 'Rating': 9},
    {'Movie Name': 'The Worlds End (2013)', 'Rating': 9},
    {'Movie Name': 'The Conjuring (2013)', 'Rating': 7},
    {'Movie Name': 'Carrie (2013)', 'Rating': 8},
    {'Movie Name': 'Don Jon (2013)', 'Rating': 8},
    {'Movie Name': 'Vive la France (2013)', 'Rating': 7},
    {'Movie Name': 'Saving Mr. Banks (2013)', 'Rating': 9},
    {'Movie Name': 'The Kings of Summer (2013)', 'Rating': 7},
    {'Movie Name': 'jOBS (2013)', 'Rating': 6},
    {'Movie Name': 'This Is the End (2013)', 'Rating': 8},
    {'Movie Name': 'Scary Movie 5 (2013)', 'Rating': 6},
    {'Movie Name': 'Pain &amp; Gain (2013)', 'Rating': 7},
    {'Movie Name': 'Bad Grandpa (2013)', 'Rating': 4},
    {'Movie Name': 'Machete Kills (2013)', 'Rating': 7},
    {'Movie Name': 'Filth (2013)', 'Rating': 9},
    {'Movie Name': 'Hot Fuzz (2007)', 'Rating': 10},
    {'Movie Name': 'The Starving Games (2013)', 'Rating': 4}]

    Would you recommend me the movie, Garden State (2004)?
    """

    examples_assistant_res = f"""
    1. Analyze movie watch history
    Dallas Buyers Club (2013) - Drama  
    World War Z (2013) - Action  
    Gravity (2013) - Sci-Fi  
    Her (2013) - Drama  
    12 Years a Slave (2013) - Drama  
    The Secret Life of Walter Mitty (2013) - Adventure  
    The Grand Budapest Hotel (2014) - Comedy  
    Riddick (2013) - Action  
    Kick-Ass 2 (2013) - Action  
    Star Trek Into Darkness (2013) - Sci-Fi  
    The Wolf of Wall Street (2013) - Drama  
    American Hustle (2013) - Drama  
    Nebraska (2013) - Drama  
    Inside Llewyn Davis (2013) - Drama  
    R.I.P.D. (2013) - Action  
    The Book Thief (2013) - Drama  
    Last Vegas (2013) - Comedy  
    The Purge (2013) - Horror  
    Runner Runner (2013) - Thriller  
    The (2013) - Unknown  
    The Worlds End (2013) - Comedy  
    The Conjuring (2013) - Horror  
    Carrie (2013) - Horror  
    Don Jon (2013) - Comedy  
    Vive la France (2013) - Comedy  
    Saving Mr. Banks (2013) - Drama  
    The Kings of Summer (2013) - Comedy  
    jOBS (2013) - Drama  
    This Is the End (2013) - Comedy  
    Scary Movie 5 (2013) - Comedy  
    Pain & Gain (2013) - Action  
    Bad Grandpa (2013) - Comedy  
    Machete Kills (2013) - Action  
    Filth (2013) - Drama  
    Hot Fuzz (2007) - Comedy  
    The Starving Games (2013) - Comedy  

    2. Calculate the weighted score for each genre
    Drama: (9+10+8+10+10+9+7+9+9)/9 = 9.0  
    Action: (7+6+7+4+7)/5 = 6.2  
    Sci-Fi: (9+7)/2 = 8.0  
    Adventure: 9/1 = 9.0  
    Comedy: (10+5+9+8+7+7+8+6+4+10+4)/11 = 6.8  
    Horror: (3+7+8)/3 = 6.0  
    Thriller: 7/1 = 7.0  
    Unknown: 9/1 = 9.0  

    3. Cluster the movies based on the genres and weighted score
    Cluster 1 (High-rated dramas and adventures):  
    - Dallas Buyers Club (2013)  
    - Her (2013)  
    - 12 Years a Slave (2013)  
    - The Secret Life of Walter Mitty (2013)  
    - The Wolf of Wall Street (2013)  
    - American Hustle (2013)  
    - Nebraska (2013)  
    - Inside Llewyn Davis (2013)  
    - The Book Thief (2013)  
    - Saving Mr. Banks (2013)  
    - jOBS (2013)  
    - Filth (2013)  
    
    Cluster 2 (High-rated sci-fi and unknown genres):  
    - Gravity (2013)  
    - Star
    
    4. Predict User's Rating Score
    To predict the rating for "Garden State (2004)," we can use the weighted scores of similar genres from the previous clustering and the user's preferences. "Garden State" is a romantic comedy-drama film. Based on the user's preferences, we can see that the user enjoys drama and comedy films.

    Using the weighted scores calculated earlier:

    Drama: 9.0
    Comedy: 6.8
    
    Since "Garden State" is a blend of both drama and comedy, we can take an average of the weighted scores of these two genres to predict the rating:

    Predicted Rating = (Drama Weighted Score + Comedy Weighted Score) / 2
    Predicted Rating = (9.0 + 6.8) / 2 = 7.9

    So, based on the user's preferences and the movie's genres, we can predict that the user would likely rate "Garden State (2004)" around 7.9 out of 10.
    """

    temp_user = df[df["UserId"]==user_id][["Movie Name","Rating"]].to_dict("records")[:-1]
    temp_user_latest = df[df["UserId"]==user_id]["Movie Name"][-1:].to_list()[0]

    print("User", user_id, "watched", temp_user_latest, "and the rating was", df[df["UserId"]==user_id]["Rating"][-1:].to_list()[0])
    print("Let's see the response from the recommender system")
    print("---")
    
    res = run(system_msg=system_msg, examples_user_prompt=examples_user_prompt, examples_assistant_res=examples_assistant_res, user_msg=f"I watched {temp_user}. Would you recommend me the movie,{temp_user_latest}?")
    return res

In [10]:
Markdown(test_run())

User 14179 watched The Gold Rush (1925) and the rating was 8
Let's see the response from the recommender system
---


    1. Analyze movie watch history
    World War Z (2013) - Action  
    Gravity (2013) - Sci-Fi  
    Captain Phillips (2013) - Biography  
    All Is Lost (2013) - Drama  
    Frozen (2013) - Animation  
    Iron Man 3 (2013) - Action  
    2 Guns (2013) - Action  
    The Secret Life of Walter Mitty (2013) - Adventure  
    The Hobbit The Desolation of Smaug (2013) - Adventure  
    Elysium (2013) - Sci-Fi  
    Pacific Rim (2013) - Action  
    Now You See Me (2013) - Thriller  
    Escape Plan (2013) - Action  
    Prisoners (2013) - Drama  
    Kick-Ass 2 (2013) - Action  
    Star Trek Into Darkness (2013) - Sci-Fi  
    Olympus Has Fallen (2013) - Action  
    Were the Millers (2013) - Comedy  
    The Wolf of Wall Street (2013) - Drama  
    American Hustle (2013) - Drama  
    Enough Said (2013) - Comedy  
    Insidious Chapter 2 (2013) - Horror  
    Last Vegas (2013) - Comedy  
    The Worlds End (2013) - Comedy  
    Monsters University (2013) - Animation  
    The Conjuring (2013) - Horror  
    Insidious (2010) - Horror  
    jOBS (2013) - Drama  
    This Is the End (2013) - Comedy  
    The Heat (2013) - Comedy  
    Hours (2013) - Drama  
    Scary Movie 5 (2013) - Comedy  
    Bad Grandpa (2013) - Comedy  
    Out of the Furnace (2013) - Drama  
    The Social Network (2010) - Drama  
    City Lights (1931) - Comedy  

    2. Calculate the weighted score for each genre
    Action: (10+8+5+7+1+7)/6 = 6.3  
    Sci-Fi: (9+8)/2 = 8.5  
    Biography: 9/1 = 9.0  
    Drama: (7+9+9+6+7)/5 = 7.6  
    Animation: (3+8)/2 = 5.5  
    Adventure: (9+9)/2 = 9.0  
    Thriller: 7/1 = 7.0  
    Comedy: (8+10+6+10+6+8+6+8+5+7+7+10+8+5+7)/15 = 7.1  
    Horror: (5+8+6)/3 = 6.3  

    3. Cluster the movies based on the genres and weighted score
    Cluster 1 (High-rated dramas and biographies):  
    - Captain Phillips (2013)  
    - All Is Lost (2013)  
    - Prisoners (2013)  
    - jOBS (2013)  
    - Hours (2013)  
    - Out of the Furnace (2013)  
    - The Social Network (2010)  

    Cluster 2 (High-rated adventures and sci-fi):  
    - Gravity (2013)  
    - The Hobbit The Desolation of Smaug (2013)  
    - Elysium (2013)  
    - Pacific Rim (2013)  

    Cluster 3 (High-rated comedies):  
    - Were the Millers (2013)  
    - The Wolf of Wall Street (2013)  
    - American Hustle (2013)  
    - Enough Said (2013)  
    - Last Vegas (2013)  
    - The Worlds End (2013)  
    - Monsters University (2013)  
    - This Is the End (2013)  

    Cluster 4 (High-rated actions and thrillers):  
    - World War Z (2013)  
    - Iron Man 3 (2013)  
    - 2 Guns (2013)  
    - Now You See Me (2013)  
    - Escape Plan (2013)  
    - Olympus Has Fallen (2013)  

    Cluster 5 (High-rated horrors and animations):  
    - Frozen (2013)  
    - Insidious Chapter 2 (2013)  
    - The Conjuring (2013)  
    - Insidious (2010)  
    - Scary Movie 5 (2013)  
    - Bad Grandpa (2013)  

    4. Predict User's Rating Score
    To predict the rating for "The Gold Rush (1925)," we can use the weighted scores of similar genres from the previous clustering and the user's preferences. "The Gold Rush" is a comedy film. Based on the user's preferences, we can see that the user enjoys comedy films.

    Using the weighted scores calculated earlier:

    Comedy: 7.1
    
    Since "The Gold Rush" is a comedy film, we can predict that the user would likely rate it higher than the average comedy rating of 7.1 out of 10.

    However, it's important to note that "The Gold Rush" is a silent film from 1925, which may not be to everyone's taste. So, while the user may enjoy comedy films, they may not necessarily enjoy silent films from the 1920s.

    Therefore, based on the user's preferences and the movie's genre and age, we cannot confidently recommend "The Gold Rush (1925)" to the user.

In [11]:
user_id = 13457 # 25344, 25309, 1377, 11793, 14424, 14807, 1443, 1007, 13198, 12044
Markdown(test_run(user_id))

User 13457 watched Burning Bright (2010) and the rating was 5
Let's see the response from the recommender system
---


    1. Analyze movie watch history
    Iron Man 3 (2013) - Action  
    Zero Dark Thirty (2012) - Drama  
    Side Effects (2013) - Thriller  
    Elysium (2013) - Sci-Fi  
    Pacific Rim (2013) - Action  
    The Hangover Part III (2013) - Comedy  
    Rise of the Guardians (2012) - Animation  
    The Great Gatsby (2013) - Drama  
    Argo (2012) - Drama  
    Star Trek Into Darkness (2013) - Sci-Fi  
    Olympus Has Fallen (2013) - Action  
    Fight Club (1999) - Drama  
    Oblivion (2013) - Sci-Fi  
    Dredd (2012) - Action  
    Hansel & Gretel Witch Hunters (2013) - Action  
    G.I. Joe Retaliation (2013) - Action  
    The Strangers (2008) - Horror  
    Shaun of the Dead (2004) - Comedy  
    This Is 40 (2012) - Comedy  
    Star Wars Episode I - The Phantom Menace (1999) - Sci-Fi  
    Star Wars Episode II - Attack of the Clones (2002) - Sci-Fi  
    Star Wars Episode III - Revenge of the Sith (2005) - Sci-Fi  
    The Matrix (1999) - Sci-Fi  
    Donnie Darko (2001) - Drama  
    Grown Ups 2 (2013) - Comedy  
    House at the End of the Street (2012) - Horror  
    The Bay (2012) - Horror  
    Chernobyl Diaries (2012) - Horror  
    Devil (2010) - Horror  
    Hot Rod (2007) - Comedy  
    Deadgirl (2008) - Horror  
    [Rec] (2007) - Horror  
    Snakes on a Plane (2006) - Action  
    [Rec] ?(2009) - Horror  
    Panic Room (2002) - Thriller  
    Deep Blue Sea (1999) - Action  

    2. Calculate the weighted score for each genre
    Action: (9+8+8+7+7+7)/6 = 7.7  
    Drama: (9+5+10+8+9+7)/6 = 8.0  
    Thriller: (6+7)/2 = 6.5  
    Sci-Fi: (9+8+7+7+6)/5 = 7.4  
    Animation: 8/1 = 8.0  
    Comedy: (6+8+4+7)/4 = 6.3  
    Horror: (8+7+7+3+1+7+7)/7 = 5.3  

    3. Cluster the movies based on the genres and weighted score
    Cluster 1 (High-rated actions and sci-fi):  
    - Iron Man 3 (2013)  
    - Elysium (2013)  
    - Pacific Rim (2013)  
    - Star Trek Into Darkness (2013)  
    - Olympus Has Fallen (2013)  
    - Dredd (2012)  
    - Hansel & Gretel Witch Hunters (2013)  
    - G.I. Joe Retaliation (2013)  
    - Snakes on a Plane (2006)  
    
    Cluster 2 (High-rated dramas):  
    - Zero Dark Thirty (2012)  
    - The Great Gatsby (2013)  
    - Argo (2012)  
    - Fight Club (1999)  
    - Donnie Darko (2001)  
    
    Cluster 3 (High-rated animation):  
    - Rise of the Guardians (2012)  
    
    Cluster 4 (High-rated comedy):  
    - The Hangover Part III (2013)  
    - Shaun of the Dead (2004)  
    - Grown Ups 2 (2013)  
    - Hot Rod (2007)  
    
    Cluster 5 (High-rated horror and thriller):  
    - The Strangers (2008)  
    - House at the End of the Street (2012)  
    - The Bay (2012)  
    - Chernobyl Diaries (2012)  
    - Devil (2010)  
    - Deadgirl (2008)  
    - [Rec] (2007)  
    - [Rec] ?(2009)  
    - Panic Room (2002)  
    - Deep Blue Sea (1999)  

    4. Predict User's Rating Score
    To predict the rating for "Burning Bright (2010)," we can use the weighted scores of similar genres from the previous clustering and the user's preferences. "Burning Bright" is a horror-thriller film. Based on the user's preferences, we can see that the user enjoys action, sci-fi, drama, and comedy films.

    Using the weighted scores calculated earlier:

    Action: 7.7
    Drama: 8.0
    Comedy: 6.3
    Horror: 5.3
    
    Since "Burning Bright" is a horror-thriller film, we can take an average of the weighted scores of these two genres to predict the rating:

    Predicted Rating = (Horror Weighted Score + Thriller Weighted Score) / 2
    Predicted Rating = (5.3 + 6.5) / 2 = 5.9

    So, based on the user's preferences and the movie's genres, we can predict that the user would likely rate "Burning Bright (2010)" around 5.9 out of 10. Therefore, we would not recommend this movie to the user.

In [12]:
user_id = 1007
Markdown(test_run(user_id))

User 1007 watched Untraceable (2008) and the rating was 9
Let's see the response from the recommender system
---


    1. Analyze movie watch history
    Dallas Buyers Club (2013) - Drama  
    World War Z (2013) - Action  
    Lone Survivor (2013) - Action  
    Frozen (2013) - Animation  
    Killing Them Softly (2012) - Crime  
    The Lone Ranger (2013) - Action  
    Transformers (2007) - Action  
    Prisoners (2013) - Crime  
    The Wolf of Wall Street (2013) - Drama  
    Dangerous Minds (1995) - Drama  
    The Purge (2013) - Horror  
    The Conjuring (2013) - Horror  
    Blue Jasmine (2013) - Drama  
    Lo imposible (2012) - Drama  
    Gran Torino (2008) - Drama  
    This Is the End (2013) - Comedy  
    The Prestige (2006) - Drama  
    Brave (2012) - Animation  
    Tangled (2010) - Animation  
    Before Midnight (2013) - Drama  
    Savages (2012) - Crime  
    The Words (2012) - Drama  
    The Breakfast Club (1985) - Drama  
    Almost Famous (2000) - Drama  
    The Lake House (2006) - Drama  
    The Iron Lady (2011) - Drama  
    Due Date (2010) - Comedy  
    Wall Street: Money Never Sleeps (2010) - Drama  
    In the Land of Women (2007) - Drama  
    The Curious Case of Benjamin Button (2008) - Drama  
    The Stepfather (2009) - Horror  
    The Edge of Love (2008) - Drama  

    2. Calculate the weighted score for each genre
    Drama: (10+8+10+7+10+8+10+9+7+9+8+10+8+10+10+8+10+10+9+9+10+6+8+10+10+9+6+9+10+8)/30 = 8.9  
    Action: (10+9+10+5+10+6)/6 = 8.3  
    Animation: (10+7)/2 = 8.5  
    Crime: (8+6+10+6)/4 = 7.5  
    Horror: (6+10)/2 = 8.0  
    Comedy: (10+6)/2 = 8.0  

    3. Cluster the movies based on the genres and weighted score
    Cluster 1 (High-rated dramas):  
    - Dallas Buyers Club (2013)  
    - The Wolf of Wall Street (2013)  
    - Blue Jasmine (2013)  
    - Lo imposible (2012)  
    - Gran Torino (2008)  
    - The Prestige (2006)  
    - Before Midnight (2013)  
    - Blue Valentine (2010)  
    - The Words (2012)  
    - The Curious Case of Benjamin Button (2008)  
    - The Lake House (2006)  
    - The Iron Lady (2011)  
    
    Cluster 2 (High-rated action and animation):  
    - World War Z (2013)  
    - Lone Survivor (2013)  
    - Frozen (2013)  
    - Transformers (2007)  
    - Brave (2012)  
    - Tangled (2010)  
    
    Cluster 3 (High-rated horror):  
    - The Purge (2013)  
    - The Conjuring (2013)  
    - The Stepfather (2009)  

    4. Predict User's Rating Score
    To predict the rating for "Untraceable (2008)," we can use the weighted scores of similar genres from the previous clustering and the user's preferences. "Untraceable" is a thriller crime film. Based on the user's preferences, we can see that the user enjoys dramas, actions, and crime films.

    Using the weighted scores calculated earlier:

    Drama: 8.9
    Action: 8.3
    Crime: 7.5
    
    Since "Untraceable" is a blend of crime and thriller, we can take an average of the weighted scores of these two genres to predict the rating:

    Predicted Rating = (Crime Weighted Score + Thriller Weighted Score) / 2
    Predicted Rating = (7.5 + 8.0) / 2 = 7.8

    So, based on the user's preferences and the movie's genres, we can predict that the user would likely rate "Untraceable (2008)" around 7.8 out of 10.

In [13]:
# End of notebook